In [1]:
import string
from Bio import Entrez
from Bio import SeqIO
import csv
import re
import pandas as pd
import os

In [2]:
cwd = os.getcwd()
newpath = str(cwd) + "/temp_folder"
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [3]:
def parseFile(file):
    '''
    This will take the original stockhom file, find out where the alignments actually start and end, remove all of the other
    information, add a column of identifiers, and then save the file as a pandas dataframe.
    '''
    characters = list(string.ascii_uppercase) #from the string package get all uppercase letters
    idcolumn = ['id         sequence\n'] #ID column that will be appended to the top of the alignment

    with open(file,"rt") as f:
        data = f.readlines() #read in the original file 

    first = 0
    last = 0
    for x in data: # find where the alignment starts by looking at the first letter of each line
        if x[0] in characters:
            break
        else :
            first = first +1    

    for x in data: #find out where the dot bracket notation is 
        if "#=GC SS_cons" in x:
            break
        else:
            last = last +1

    new_data = idcolumn + data[first:last] #trim the alignment out of the file           
    
    with open(str(newpath)+ "/"+ "temp.txt","w") as f: #write a new file that does not have the alignment stats and make a temp file
        for x in new_data:
            if x[0] != "#" :
                f.write(x)
            else:
                pass
    
    df = pd.read_csv(str(newpath)+ "/"+ "temp.txt", delim_whitespace=True)
    
    return df

In [4]:
def getNames(df):

    id_number_column = df['id'] # find id column and set it as a variable
    query_list = [] #Temp list holding all of the acession number
    number_column = [] #range of sequences

    for s in id_number_column:
        sep = s.split("/")

        query_list.append(sep[0]) #append the acession number
        number_column.append(sep[1]) #append the sequence range

    df["acession_number"] = query_list #make a new column with the acession number
    df["sequence_range"] = number_column # make a new column with the sequence range

    return df


In [5]:
def getNamesRscape(df):

    id_number_column = df['id']
    query_list = []
    number_column = []
    for s in id_number_column:
        if s.startswith('NC'):
            s_split = s.split('_')
            combined = s_split[0] + '_' + s_split[1]
            query_list.append(combined)
            number_column.append(s_split[2])
        else:
            s_split = s.split('_')
            query_list.append(s_split[0])
            number_column.append(s_split[1])
        
    df["acession_number"] = query_list #make a new column with the acession number
    df["sequence_range"] = number_column # make a new column with the sequence range
    
    return df


In [6]:
# initialize some default parameters
Entrez.email = "matthew.j.szucs@gmail.com" # add email address
db = 'nucleotide'  #set the database to nucleotide      

def getNameForNucleotide(id_number):
    handle = Entrez.efetch(db='nucleotide', id =id_number, retmode = "xml") # Do the search
    record = Entrez.read(handle) # read the search results
    
    name = record[0]["GBSeq_source"] # extract the name of the searched entry
    name_split = name.split() # split the name up into multiple strings by space
    name_join = "_".join(name_split) # join the strings by "_"
    return name_join

In [7]:
def entrezQuery(df,email,db = 'nucleotide'):
    '''
    This function will run through the list of acession numbers and search the NCBI, get the entry and parse out the organism name and store it in its own
    column. This will take a long amount of time
    input: dataframe from getNames()
    output: dataframe with a new column holding the organism name
    
    '''
    Entrez.email = str(email) # add email address
    db = str(db)  #set the database to nucleotide 
    query_list = df["acession_number"].values.tolist()
    query_results_list = []
    i = 0
    for id_value in query_list:
        cur_name = getNameForNucleotide(id_value)
        query_results_list.append(cur_name)
        i = i+1
        print("Done with", i ,"out of", len(query_list), "items")
    
    df["names"] = query_results_list
    
    return df


In [8]:
def mergeColumns(df):
    '''
    This will make a new id which is a concatination of all of these columns that were made. 
    input: df with names from entrezQuery()
    output: df with the concatinated names
    '''
    df["name_accession_range"] = df['names'].astype(str) +"_"+ df["acession_number"].astype(str) + "_" + df["sequence_range"]
    
    return(df)
    

In [9]:
def addWhitespace(df):
    '''
    This will look at the dataframe and determine how many spaces to make with the final alignment. 
    input: dataframe from mergeColumns()
    output: new DF and also a temporary file of the alignment. 
    '''
    final_file_str = ''
 
    max_num_characters = 0
    for s in df["name_accession_range"].values.tolist(): #look at the list of values and determine the longest string
        temp = len(s)
        if temp > max_num_characters:
            max_num_characters = temp #save the maximum length name string
            
    max_characters = max_num_characters + 8 # take that value and then add 8 white spaces
            
    for ind in range(0, len(df)):
    
        # Get the query results, append them to the front, 
        results = df.iloc[ind]["name_accession_range"]
        rna_sequence = df.iloc[ind]['sequence']

        # Get the number of spaces
        space_str = ' ' * (max_characters - len(results))

        # Add the values to our file
        final_file_str+= f'{results}{space_str}{rna_sequence}\n'

    text_file = open(str(newpath)+ "/"+ "query_and_results_temp.txt", "w") #save the file as a temp file. 
    n = text_file.write(final_file_str) # write the temporary file
    text_file.close()
    
    return df

In [10]:
def addOtherStockholmInfo(file, temp_file = str(newpath)+ "/"+ "query_and_results_temp.txt"):
    '''
    This will take the alignment temporary file, and then strip out all of the relevent information and then add it to a new alignment file. 
    input: temporary file, original stockholm file
    output: final alignment. 
    '''
    characters = list(string.ascii_uppercase) #from the string package get all uppercase letters

    with open(file,"rt") as f:
        data = f.readlines()
    
    with open(temp_file,"rt") as f2:
        temp = f2.readlines()
    
    first = 0
    last = 0
    for x in data:
        if x[0] in characters:
            break
        else :
            first = first +1    

    for x in data:
        if "#=GC SS_cons" in x:
            break
        else:
            last = last +1

    top = data[0:first]
    bottom = data[last:len(data)]
    
    text_file = open("finalStockholm.txt", "w")
    text_file.writelines(top)
    text_file.writelines(temp)
    text_file.writelines(bottom)
    text_file.close()
    
    return 0

In [ ]:
a = parseFile("221107_Class2_xrRNA_FullViralGenomemultiple_align.sto")
b = getNames(a)
c = entrezQuery(b, "matthew.szucs@gmail.com")


Done with 1 out of 6830 items
Done with 2 out of 6830 items
Done with 3 out of 6830 items
Done with 4 out of 6830 items
Done with 5 out of 6830 items
Done with 6 out of 6830 items
Done with 7 out of 6830 items
Done with 8 out of 6830 items
Done with 9 out of 6830 items
Done with 10 out of 6830 items
Done with 11 out of 6830 items
Done with 12 out of 6830 items
Done with 13 out of 6830 items
Done with 14 out of 6830 items
Done with 15 out of 6830 items
Done with 16 out of 6830 items
Done with 17 out of 6830 items
Done with 18 out of 6830 items
Done with 19 out of 6830 items
Done with 20 out of 6830 items
Done with 21 out of 6830 items
Done with 22 out of 6830 items
Done with 23 out of 6830 items
Done with 24 out of 6830 items
Done with 25 out of 6830 items
Done with 26 out of 6830 items
Done with 27 out of 6830 items
Done with 28 out of 6830 items
Done with 29 out of 6830 items
Done with 30 out of 6830 items
Done with 31 out of 6830 items
Done with 32 out of 6830 items
Done with 33 out 

In [ ]:
b.head()